In [1]:
with open("requirements.txt", "w") as f:
    f.write("kfp==1.4.0\n")
    
!pip install -r requirements.txt  --upgrade --user

In [2]:
import kfp.dsl as dsl
import kfp
from kfp import components

kfserving_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/kfserving/component.yaml')

@dsl.pipeline(
  name='KFServing pipeline',
  description='A pipeline for KFServing.'
)
def kfservingPipeline(
    action='apply',
    model_name='tensorflow-sample',
    model_uri='gs://kfserving-samples/models/tensorflow/flowers',
    namespace='kubeflow-user',
    framework='tensorflow'):

    kfserving = kfserving_op(action = action,
                             model_name=model_name,
                             model_uri=model_uri,
                             namespace=namespace,
                             framework=framework).set_image_pull_policy('Always')



In [3]:
# Compile pipeline
kfp.compiler.Compiler().compile(kfservingPipeline, 'tf-flower.zip')